# Análisis de Tweets

- Mario Horacio Garrido Czacki
- E. David Guzmán Ramírez

Generar un identificador de tendencias de tweets, para la descarga no se usarán filtros, sin
embargo, los tweets deberán contener las palabras de ’UNAM’, ’virus’, ’hospital’,’covid-19’. Debes graficar las primeras 10 tendencias por día y agrupados durante todo el periodo de
recolección.

In [7]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import warnings
# warnings.filterwarnings("ignore")

## Keys y tokens

Ponemos nuestras llaves y tokens para usar la API de Twitter:

In [10]:
API_key = ''
API_secret_key = ''
Access_token = ''
Access_token_secret = ''

In [11]:
auth = tweepy.OAuthHandler(API_key, API_secret_key)
auth.set_access_token(Access_token, Access_token_secret)

api = tweepy.API(auth)

Ahora empezamos a descargar tweets, cada status tiene la siguiente información, sólo guardaremos la que nos interesa

```
['_api', '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'display_text_range', 'entities', 'extended_tweet', 'favorite', 'favorite_count', 'favorited', 'filter_level', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'parse', 'parse_list', 'place', 'possibly_sensitive', 'quote_count', 'reply_count', 'retweet', 'retweet_count', 'retweeted', 'retweets', 'source', 'source_url', 'text', 'timestamp_ms', 'truncated', 'user']
```

## Descarga

Si hacemos las descargas sin filtros nos salen tweets de todo el mundo y no salen algunas de las palabras que nos gustaría (sobre todo UNAM), por lo que restringimos la búsqueda a México, para poder filtrar por ubicación podemos poner 'cajas delimitadoras' en los parámetros (ver [BoundingBox](https://boundingbox.klokantech.com/) para sacar las cajas).

In [4]:
tweets_mayo_24 = pd.DataFrame(columns = ['created_at', 'text', 'coordinates', 'hashtags'])
tweets = []

#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    
    def on_status(self, status):
        
        # Tomamos 25,000 tweets
        if tweets_mayo_24.shape[0] < 25000:
            tweet = [status.created_at, status.text, status.coordinates, status.entities['hashtags']]
            tweets_mayo_24.loc[tweets_mayo_24.shape[0]] = tweet
            
#             # Es buena idea guardarlos de vez en cuando por si pasa algo
#             if tweets_mayo_24.shape[0]%1000 == 0:
#                 print(tweets_mayo_24.shape[0])
#                 tweets_mayo_24.to_csv('Datos/tweets_mayo_24_2_David.csv', index = False)
        else:
            print('FIN!')
            return False
        
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

# Centro de México
caja1 = [-113.19,18.51,-86.23,24.94]
# Sur de México
caja2 = [-104.5743,14.7939,-92.8739,19.3242]
# Norte de México, salen muchos en inglés así que no se usará
caja3 = [-109.19,24.56,-103.72,28.73]

%time myStream.filter(locations=[-113.19,18.51,-86.23,24.94,-104.5743,14.7939,-92.8739,19.3242])

500
FIN!
CPU times: user 2.12 s, sys: 23.1 ms, total: 2.14 s
Wall time: 7min 39s


In [18]:
tweets_mayo_24  = pd.read_csv('Datos/David/tweets_mayo_24_David.csv')

In [21]:
tweets_mayo_24.head()

,created_at,text,coordinates,hashtags
0,2020-05-24 07:07:42,@Max91683268 Manda MD,NaN,[]
1,2020-05-24 07:07:42,Jalisco got some pretty females,NaN,[]
2,2020-05-24 07:07:43,@unamariant Somos dos 💅,NaN,[]
3,2020-05-24 07:07:44,@KikaBLUE2 Qué rica,NaN,[]
4,2020-05-24 07:07:47,Bájenle ....se llama ACOSO a la joven Secreta...,NaN,[]


In [14]:
help(myStream.filter)

Help on method filter in module tweepy.streaming:

filter(follow=None, track=None, is_async=False, locations=None, stall_warnings=False, languages=None, encoding='utf8', filter_level=None) method of tweepy.streaming.Stream instance



In [ ]:
tweets_mayo_24.to_csv('Datos/David/tweets_mayo_24_David.csv', index = False)

## Referencias/Ayuda

- [Streaming With Tweepy](http://docs.tweepy.org/en/latest/streaming_how_to.html)
- [Filter realtime Tweets](https://developer.twitter.com/en/docs/tweets/filter-realtime/guides/basic-stream-parameters)
- [Tweet objects](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)
- [Filtered stream v1 (FAQ)](https://developer.twitter.com/en/docs/labs/filtered-stream/faq)
- [How to add a location filter to tweepy module](https://stackoverflow.com/questions/22889122/how-to-add-a-location-filter-to-tweepy-module)
- [Filter Twitter feeds only by language](https://stackoverflow.com/questions/26890605/filter-twitter-feeds-only-by-language)
- [How can I use tweepy without a filter](https://stackoverflow.com/questions/27530269/how-can-i-use-tweepy-without-a-filter)